In [4]:
import os
import numpy as np 
import pandas as pd
import arcpy
from arcpy import env

In [3]:
result_path = r"D:\ArcGISProjects\workspace\duyun\prediction_table\prediction_ph.csv"

In [5]:
# 读取预测结果
data = pd.read_csv(result_path,index_col=False)

In [7]:
stander_path = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb\DEM"

In [6]:
data.columns

Index(['prediction_0', 'prediction_1', 'prediction_2', 'prediction_3',
       'prediction_4', 'prediction_5', 'prediction_6', 'prediction_7',
       'prediction_8', 'prediction_9',
       ...
       'prediction_326', 'prediction_327', 'prediction_328', 'prediction_329',
       'prediction_330', 'prediction_331', 'prediction_332', 'prediction_333',
       'prediction_334', 'prediction_335'],
      dtype='object', length=336)

In [12]:
# 合并除第一列外的所有列到第一列
frames = []
for col in data.columns:
    frames.append(data[col])

# 合并 DataFrame
merged_data = pd.concat(frames, axis=0, ignore_index=True)

In [16]:
merged_data.dropna(inplace=True)

In [18]:
merged_data.size

167675040

In [20]:
type(merged_data)

pandas.core.series.Series

In [15]:
dem_array = arcpy.RasterToNumPyArray(stander_path)
dem_array.shape

(12720, 13182)

In [19]:
dem_array.shape[0] * dem_array.shape[1]

167675040

In [21]:
raster_array = np.reshape(merged_data.values,dem_array.shape)
raster_array

array([[5.7381   , 5.7215667, 5.675367 , ..., 6.6028666, 6.6084   ,
        6.5070667],
       [5.7678666, 5.767367 , 5.701267 , ..., 6.663767 , 6.5830336,
        6.540867 ],
       [5.6937   , 5.7272334, 5.8924   , ..., 6.6605334, 6.6582665,
        6.634267 ],
       ...,
       [5.6154   , 5.3858333, 5.4633665, ..., 5.3565664, 5.3819   ,
        5.420133 ],
       [5.6188664, 5.4785333, 5.789767 , ..., 5.3862667, 5.364333 ,
        5.391033 ],
       [5.579467 , 5.4792333, 5.6878   , ..., 5.405    , 5.3732   ,
        5.4061666]])

In [24]:
env.workspace = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb"
env.extent = "DEM"

In [25]:
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [26]:
# 按掩膜处理
result_raster = mask_raster(raster_array,"DEM", 5)
result_raster.save("prediction_ph")
print("完成")

完成
